[Reference](https://leapcell.medium.com/fastapi-is-overkill-starlette-and-pydantic-are-all-you-really-need-2b2d55c53de0)

# Asynchronous Request Handling

In [2]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00


In [3]:
from fastapi import FastAPI
import asyncio

# Create a FastAPI application instance
app = FastAPI()
# Use a decorator to define a GET request route. The function is an asynchronous function and can handle time-consuming operations without blocking other requests
@app.get("/async_items/")
async def async_read_items():
    await asyncio.sleep(1)  # Simulate an I/O operation and pause for 1 second
    return {"message": "FastAPI asynchronous processing example"}

In [4]:
from starlette.applications import Starlette
from starlette.responses import JSONResponse
import asyncio

# Create a Starlette application instance
app = Starlette()

# Directly define the route on the application instance, specifying the path and request method. The handling function is an asynchronous function
@app.route("/async_items/", methods=["GET"])
async def async_read_items(request):
    await asyncio.sleep(1)  # Simulate an I/O operation and pause for 1 second
    return JSONResponse({"message": "Starlette asynchronous processing example"})

# Use of Middleware

In [5]:
from starlette.applications import Starlette
from starlette.responses import JSONResponse
from starlette.middleware.base import BaseHTTPMiddleware
import logging

# Configure the logger
logger = logging.getLogger(__name__)

# Custom logging middleware, inheriting from BaseHTTPMiddleware
class LoggingMiddleware(BaseHTTPMiddleware):
    async def dispatch(self, request, call_next):
        # Log the request information, including the request method and URL
        logger.info(f"Request: {request.method} {request.url}")
        # Continue to process the request and get the response
        response = await call_next(request)
        # Log the response status code
        logger.info(f"Response: {response.status_code}")
        return response

# Create a Starlette application instance and pass in the middleware instance
app = Starlette(middleware=[LoggingMiddleware(app)])

# Define the route handling function
@app.route("/middleware_example/", methods=["GET"])
async def middleware_example(request):
    return JSONResponse({"message": "The middleware is in effect"})

# WebSocket Support

In [6]:
from starlette.applications import Starlette
from starlette.websockets import WebSocket, WebSocketDisconnect
import json

# Create a Starlette application instance
app = Starlette()

# Store the WebSocket objects of connected clients
connected_clients = []

# Define the WebSocket route handling function
@app.websocket_route("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()  # Accept the WebSocket connection
    connected_clients.append(websocket)  # Add the connected client to the list
    try:
        while True:
            # Receive the text data sent by the client
            data = await websocket.receive_text()
            message = json.loads(data)  # Parse the received JSON string into a Python object
            for client in connected_clients:
                if client != websocket:
                    # Forward the message to other clients except the sender
                    await client.send_text(json.dumps(message))
    except WebSocketDisconnect:
        connected_clients.remove(websocket)  # Remove the client from the list when the connection is disconnected

# Data Validation and Serialization

In [7]:
from fastapi import FastAPI
from pydantic import BaseModel

# Create a FastAPI application instance
app = FastAPI()

# Use Pydantic to define a data model for validating and serializing data
class Item(BaseModel):
    name: str
    price: float

# Define the route handling function. FastAPI will automatically validate the incoming data and serialize the response
@app.post("/fastapi_items/")
async def create_fastapi_item(item: Item):
    return item

In [8]:
from starlette.applications import Starlette
from starlette.responses import JSONResponse
from starlette.requests import Request
from pydantic import BaseModel

# Create a Starlette application instance
app = Starlette()

# Use Pydantic to define a data model for validating and serializing data
class Item(BaseModel):
    name: str
    price: float

# Define the route handling function and manually handle the request data and validation logic
@app.route("/starlette_items/", methods=["POST"])
async def create_starlette_item(request: Request):
    data = await request.json()  # Get the JSON data from the request
    try:
        item = Item(**data)  # Use Pydantic to validate the data. If it is not valid, an exception will be thrown
    except ValueError as e:
        return JSONResponse({"error": str(e)}, status_code=400)  # Return an error response if the validation fails
    return JSONResponse(item.dict())  # Return the serialized response if the validation passes

# Complex Data Models and Nested Validation

In [9]:
from starlette.applications import Starlette
from starlette.responses import JSONResponse
from starlette.requests import Request
from pydantic import BaseModel

# Create a Starlette application instance
app = Starlette()
# Define the address data model
class Address(BaseModel):
    street: str
    city: str
    zip_code: str

# Define the user data model, which contains a nested address model
class User(BaseModel):
    username: str
    email: str
    address: Address

# Define the route handling function to handle the validation and storage of user data
@app.route("/users/", methods=["POST"])
async def create_user(request: Request):
    data = await request.json()  # Get the JSON data from the request
    try:
        user = User(**data)  # Use Pydantic to validate the nested data. If it is not valid, an exception will be thrown
    except ValueError as e:
        return JSONResponse({"error": str(e)}, status_code=400)  # Return an error response if the validation fails
    return JSONResponse(user.dict())  # Return the serialized response if the validation passes

# Deep Integration of Starlette and Pydantic

In [10]:
from starlette.applications import Starlette
from starlette.responses import JSONResponse
from starlette.requests import Request
from starlette.exceptions import HTTPException
from starlette.middleware.cors import CORSMiddleware
from pydantic import BaseModel

# Create a Starlette application instance
app = Starlette()
# Add CORS middleware to allow requests from all origins (in a production environment, specific domain names should be restricted)
app.add_middleware(CORSMiddleware, allow_origins=["*"])
# Use Pydantic to define the product data model
class Product(BaseModel):
    name: str
    price: float
    quantity: int
# List to store product data
products = []

# Define the route handling function for creating products
@app.route("/products/", methods=["POST"])
async def create_product(request: Request):
    data = await request.json()  # Get the JSON data from the request
    try:
        product = Product(**data)  # Use Pydantic to validate the data. If it is not valid, an exception will be thrown
    except ValueError as e:
        raise HTTPException(status_code=400, detail=str(e))  # Return an HTTP exception if the validation fails
    products.append(product.dict())  # Add the product data to the list if the validation passes
    return JSONResponse(product.dict())  # Return the created product data

# Define the route handling function for getting all products
@app.route("/products/", methods=["GET"])
async def get_products(request):
    return JSONResponse(products)  # Return all product data